In [ ]:
import sys
#np.set_printoptions(threshold=sys.maxsize)
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter
from utils2 import sigmoid, get_batches, compute_pca, get_dict

nltk.download('punkt')


In [ ]:
# Load, tokenize and process the data
# Download sentence tokenizer
nltk.data.path.append('.')
with open('./data/shakespeare.txt') as f:
    data = f.read()
data = re.sub(r'[,.!?;-]', '',data)
data = nltk.word_tokenize(data)
data = [ ch.lower() for ch in data if ch.isalpha()]
print("Number of tokens:", len(data),'\n', data[:20])

In [ ]:
# Compute the frequency distribution of the words in the dataset (vocabulary)
fdist = nltk.FreqDist(word for word in data)
print("Size of vocabulary: ",len(fdist) )
print("Most frequent tokens: ",fdist.most_common(20) )

In [ ]:
# get_dict creates two dictionaries, converting words to indices and viceversa.
word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)
print("Size of vocabulary: ", V)

In [ ]:
def initialize_model(N,V, random_seed=1):
    '''
    Inputs: 
        N:  dimension of hidden vector 
        V:  dimension of vocabulary
        random_seed: random seed for consistent results in the unit tests
     Outputs: 
        W1, W2, b1, b2: initialized weights and biases
    '''
    
    np.random.seed(random_seed)
    # W1 has shape (N,V)
    W1 = np.random.rand(N, V)
    
    # W2 has shape (V,N)
    W2 = np.random.rand(V, N)
    
    # b1 has shape (N,1)
    b1 = np.random.rand(N, 1)
    
    # b2 has shape (V,1)
    b2 = np.random.rand(V, 1)
    
    return W1, W2, b1, b2

def softmax(z):
    '''
    Inputs: 
        z: output scores from the hidden layer
    Outputs: 
        yhat: prediction (estimate of y)
    '''
    # Calculate yhat (softmax)
    yhat = np.exp(z) / np.sum(np.exp(z), axis=0)

    return yhat

def relu(z):
    result = z.copy()
    result[result < 0] = 0
    return result

def forward_prop(x, W1, W2, b1, b2):
    '''
    Inputs: 
        x:  average one hot vector for the context 
        W1, W2, b1, b2:  matrices and biases to be learned
     Outputs: 
        z:  output score vector
    '''

    # Calculate h
    h = np.dot(W1, x) + b1
  
    # Apply the relu on h, 
    # store the relu in h
    h = relu(h)
    
    # Calculate z
    z = np.dot(W2, h) + b2

    return z, h

# compute_cost: cross-entropy cost function
def compute_cost(y, yhat, batch_size):
    
    # cost function
    epsilon = 0.000001
    logprobs = np.multiply(np.log(yhat + epsilon), y)
    cost = -1 / batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    
    return cost

def back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size):
    '''
    Inputs: 
        x:  average one hot vector for the context 
        yhat: prediction (estimate of y)
        y:  target vector
        h:  hidden vector (see eq. 1)
        W1, W2, b1, b2:  matrices and biases  
        batch_size: batch size 
     Outputs: 
        grad_W1, grad_W2, grad_b1, grad_b2:  gradients of matrices and biases   
    '''
    # Compute l1 as W2^T (Yhat - Y)
    # and re-use it whenever you see W2^T (Yhat - Y) used to compute a gradient
    l1 = np.dot(W2.T, yhat - y)

    # Apply relu to l1
    Sl1 = relu(l1)

    # compute the gradient for W1
    grad_W1 = np.dot(l1, x.T) / batch_size

    # Compute gradient of W2
    grad_W2 = np.dot(yhat - y, h.T) / batch_size
    
    # compute gradient for b1
    grad_b1 = np.sum(l1, axis=1, keepdims=True) / batch_size

    # compute gradient for b2
    grad_b2 = np.sum(yhat - y, axis=1, keepdims=True) / batch_size
    
    return grad_W1, grad_W2, grad_b1, grad_b2

def accuracy(y, y_hat):
    return np.sum(y==y_hat)/len(y)

def print_acc(W1, W2, b1, b2):
    print()
    x, y = next(get_batches(data, word2Ind, V, C, len(data)-4))
    z, h = forward_prop(x, W1, W2, b1, b2)
    yhat = softmax(z)
    s = np.sum(yhat[:,0] == yhat[:,1])
    if s != 0:
        print("Error s != 0")

    s = y * yhat
    print("y*yhat: " + str(np.sum(s)))
    
    yhat_ind = np.argmax(yhat, axis=0)
    y_ind = np.argmax(y, axis=0)
    
    acc = accuracy(y_ind, yhat_ind)
    
    print("Accuracy: " + str(acc))
    print()
    
def save_weights_to_disk(W1, W2, b1, b2, alpha='', batch_size=''):
    print("Saving weights to disk")
    np.savetxt(f"W1_{alpha}_{batch_size}.csv", W1, delimiter=',')
    np.savetxt(f"W2_{alpha}_{batch_size}.csv", W2, delimiter=',')
    np.savetxt(f"b1_{alpha}_{batch_size}.csv", b1, delimiter=',')
    np.savetxt(f"b2_{alpha}_{batch_size}.csv", b2, delimiter=',')

def load_weights_from_disk(alpha='', batch_size=''):
    print("Loading weights from disk")
    W1 = np.loadtxt(f"W1_{alpha}_{batch_size}.csv", delimiter=',')
    W2 = np.loadtxt(f"W2_{alpha}_{batch_size}.csv", delimiter=',')
    b1 = np.expand_dims(np.loadtxt(f"b1_{alpha}_{batch_size}.csv", delimiter=','), axis=1)
    b2 = np.expand_dims(np.loadtxt(f"b2_{alpha}_{batch_size}.csv", delimiter=','), axis=1)
    
    return W1, W2, b1, b2

In [ ]:
def gradient_descent(W1, W2, b1, b2, data, word2Ind, N, V, num_iters, batch_size, alpha, 
                     alpha_decay=False, random_seed=282, initialize_model=initialize_model, 
                     get_batches=get_batches, forward_prop=forward_prop, 
                     softmax=softmax, compute_cost=compute_cost, 
                     back_prop=back_prop):
    
    '''
    This is the gradient_descent function
    
      Inputs: 
        data:      text
        word2Ind:  words to Indices
        N:         dimension of hidden vector  
        V:         dimension of vocabulary 
        num_iters: number of iterations  
        random_seed: random seed to initialize the model's matrices and vectors
        initialize_model: your implementation of the function to initialize the model
        get_batches: function to get the data in batches
        forward_prop: your implementation of the function to perform forward propagation
        softmax: your implementation of the softmax function
        compute_cost: cost function (Cross entropy)
        back_prop: your implementation of the function to perform backward propagation
     Outputs: 
        W1, W2, b1, b2:  updated matrices and biases after num_iters iterations

    '''
    
    iters = 0
    C = 2 
    total_cost = 0
    
    x_val, y_val = next(get_batches(data[:104], word2Ind, V, C, 100))
    
    for x, y in get_batches(data[104:], word2Ind, V, C, batch_size):
        # get z and h
        z, h = forward_prop(x, W1, W2, b1, b2)

        # get yhat
        yhat = softmax(z)
        
        # get cost
        cost = compute_cost(y, yhat, batch_size)
        total_cost += cost
  
        # get gradients
        grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size)
        
        # update weights and biases
        W1 = W1 - alpha * grad_W1
        W2 = W2 - alpha * grad_W2
        b1 = b1 - alpha * grad_b1
        b2 = b2 - alpha * grad_b2

        ### END CODE HERE ###
        iters +=1
        
        if iters % 100 == 0:
            z_val, h_val = forward_prop(x_val, W1, W2, b1, b2)
            yhat_val = softmax(z_val)
            c = compute_cost(y_val, yhat_val, 100)
            
            print(f"   iteration: {iters}")
            print(f"   alpha: {alpha:.6f}")
            print(f"   val cost:   {c:.6f}")
            print(f"   train cost: {total_cost / iters:.6f}")
            print()
        
        if iters % 1000 == 0:
            print_acc(W1, W2, b1, b2)
            save_weights_to_disk(W1, W2, b1, b2, alpha=str(alpha), batch_size=str(batch_size))
        
        if iters == num_iters:
            print()
            print(f"total train cost: {total_cost / num_iters:.6f}")
            break
        
        if iters % 1000 == 0:
            if alpha_decay:
                alpha *= 0.9
            
    return W1, W2, b1, b2

In [ ]:
C = 2
N = 512
word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)

num_iters = 10000
batch_size = 256
alpha = .3
alpha_decay = False

In [ ]:
W1, W2, b1, b2 = initialize_model(N,V, random_seed=42) #W1=(N,V) and W2=(V,N)
#W1, W2, b1, b2 = load_weights_from_disk(alpha='0.3', batch_size='256')
#print_acc(W1, W2, b1, b2)

print("Call gradient_descent")
W1, W2, b1, b2 = gradient_descent(W1, W2, b1, b2, data, word2Ind, N, V, num_iters, batch_size, alpha=alpha, alpha_decay=alpha_decay)

weights = (W1.copy(), W2.copy(), b1.copy(), b2.copy())


In [ ]:
save_weights_to_disk(W1, W2, b1, b2)

In [ ]:
#alpha: 0.1
#y*yhat: 1020.9995649051946
#Accuracy: 0.08372444823707381
#total train cost: 6.540943

In [ ]:
#alpha: 0.3

#batch size: 128
#val cost:   6.883895
#train cost: 6.108548
#y*yhat: 1641.483304054504
#Accuracy: 0.11704355921027113

#batch size: 256
#val cost:   6.937128
#train cost: 6.073951
#y*yhat: 1855.7868529756695
#Accuracy: 0.12664365230208294

In [ ]:
#alpha: 0.500000
#val cost:   6.823716
#train cost: 6.049268
#y*yhat: 1548.4969183230244
#Accuracy: 0.10965439664869478

In [ ]:
#alpha: 1.0
#val cost:   6.777105
#train cost: 6.102779
#y*yhat: 1172.2643192233247
#Accuracy: 0.09115239905356658

In [ ]:
# visualizing the word vectors here
from matplotlib import pyplot
%config InlineBackend.figure_format = 'svg'
words = ['king', 'queen','lord','man', 'woman','dog','wolf',
         'rich','happy','sad']

embs = (W1.T + W2)/2.0
 
# given a list of words and the embeddings, it returns a matrix with all the embeddings
idx = [word2Ind[word] for word in words]
X = embs[idx, :]
print(X.shape, idx)  # X.shape:  Number of words of dimension N each 

In [ ]:
result= compute_pca(X, 2)
pyplot.scatter(result[:, 0], result[:, 1])
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

In [ ]:
result= compute_pca(X, 4)
pyplot.scatter(result[:, 3], result[:, 1])
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 3], result[i, 1]))
pyplot.show()